In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

In [4]:
!ls 'drive/My Drive/Hackathon/Hackathon/SentimentAnalysis'

review_test.csv  review_train.csv


In [0]:
sent_train = pd.read_csv('drive/My Drive/Hackathon/Hackathon/SentimentAnalysis/review_train.csv')
sent_test = pd.read_csv('drive/My Drive/Hackathon/Hackathon/SentimentAnalysis/review_test.csv')

In [6]:
sent_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14825 entries, 0 to 14824
Data columns (total 3 columns):
Text         14825 non-null object
Score        14825 non-null int64
Sentiment    14825 non-null int64
dtypes: int64(2), object(1)
memory usage: 347.5+ KB


In [7]:
sent_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3707 entries, 0 to 3706
Data columns (total 3 columns):
Text         3707 non-null object
Score        3707 non-null int64
Sentiment    3707 non-null int64
dtypes: int64(2), object(1)
memory usage: 87.0+ KB


In [0]:
sent_train['src'] = 0
sent_test['src'] = 1

In [0]:
src = pd.concat([sent_train,sent_test])

In [10]:
src.head()

,Text,Score,Sentiment,src
0,"I got a free sample of these once, and now--we...",5,1,0
1,I used to get this Tea when I lived in Washing...,4,1,0
2,This is my all time favorite 'grab and go' sna...,5,1,0
3,This flavor is very good and unexpected. The ...,4,1,0
4,thrilled to have this assortment as i got the ...,4,1,0


In [11]:
src['Sentiment'].unique()

array([1, 0])

In [12]:
import unicodedata

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

import re

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text
  
!pip install contractions
!pip install textsearch
import contractions
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
wnl = WordNetLemmatizer()

def pos_tag_wordnet(tagged_tokens):
    tag_map = {'j': wordnet.ADJ, 'v': wordnet.VERB, 'n': wordnet.NOUN, 'r': wordnet.ADV}
    new_tagged_tokens = [(word, tag_map.get(tag[0].lower(), wordnet.NOUN))
                            for word, tag in tagged_tokens]
    return new_tagged_tokens

def wordnet_lemmatize_text(text):
    tagged_tokens = nltk.pos_tag(nltk.word_tokenize(text))
    wordnet_tokens = pos_tag_wordnet(tagged_tokens)
    lemmatized_text = ' '.join(wnl.lemmatize(word, tag) for word, tag in wordnet_tokens)
    return lemmatized_text
  
def remove_stopwords(text, is_lower_case=False, stopwords=None):
    if not stopwords:
        stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

     |████████████████████████████████| 317kB 4.9MB/s 
     |████████████████████████████████| 245kB 48.9MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81700 sha256=6967c93cce4269713908f81a7958b3b5fe1d3dd179c34dd4c3c930e6f7fb9aff
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
col = 'Text'
src[col] = src[col].map(lambda x: str(x).lower())
src[col] = src[col].map(lambda x: remove_accented_chars(x))
src[col] = src[col].map(lambda x: remove_special_characters(x))
src[col] = src[col].map(lambda x: contractions.fix(x))
src[col] = src[col].map(lambda x: wordnet_lemmatize_text(x))
src[col] = src[col].map(lambda x: remove_stopwords(x,True))

In [14]:
src

,Text,Score,Sentiment,src
0,get free sample nowwere subscribe save program...,5,1,0
1,use get tea live washington state miss niece t...,4,1,0
2,time favorite grab go snack bar leave sugar ru...,5,1,0
3,flavor good unexpected flavor cheese garlic go...,4,1,0
4,thrill assortment get chance choose another de...,4,1,0
5,love flavor coffee drawback set large cup want...,4,1,0
6,like last review mixture dog 2 chihuahua eat a...,5,1,0
7,wonderful handy product I thrill yummy way use...,5,1,0
8,great value sure place bag crowd transfer hard...,5,1,0
9,like flavor kind reminds jello pudding bite th...,4,1,0


In [0]:
train = src[src['src'] == 0]
test = src[src['src'] == 1]

**Dont use score below example use score and predicts 100%**

In [0]:
X_train = train[['Score','src']]
Y_train = train['Sentiment']
X_test = test[['Score','src']]
Y_test = test['Sentiment']

In [0]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import classification_report

In [45]:
from sklearn import  linear_model
mdl = linear_model.LogisticRegressionCV()
mdl.fit(X_train,Y_train)
Y_pred=mdl.predict(X_test)
print(metrics.confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test, Y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[[ 592    0]
 [   0 3115]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       592
           1       1.00      1.00      1.00      3115

    accuracy                           1.00      3707
   macro avg       1.00      1.00      1.00      3707
weighted avg       1.00      1.00      1.00      3707



In [0]:
X_train = train['Text']
Y_train = train['Sentiment']
X_test = test['Text']
Y_test = test['Sentiment']

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
#grid search result

In [0]:
vectorizer = TfidfVectorizer(ngram_range=(1,2),min_df=5,max_df=1.0,use_idf=True, sublinear_tf=True) 
counts = vectorizer.fit_transform(X_train)
vocab = vectorizer.vocabulary_
classifier = SGDClassifier(alpha=1e-05,max_iter=50,penalty='elasticnet')
targets = Y_train
classifier = classifier.fit(counts, targets)
example_counts = vectorizer.transform(X_test)
Y_pred = classifier.predict(example_counts)

In [63]:
print(metrics.confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test, Y_pred))

[[ 383  209]
 [ 142 2973]]
              precision    recall  f1-score   support

           0       0.73      0.65      0.69       592
           1       0.93      0.95      0.94      3115

    accuracy                           0.91      3707
   macro avg       0.83      0.80      0.82      3707
weighted avg       0.90      0.91      0.90      3707



In [0]:
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

In [65]:
for mdl in [ensemble.AdaBoostClassifier(),tree.DecisionTreeClassifier(),XGBClassifier(),naive_bayes.BernoulliNB()]:
   classifier = mdl  
   targets = Y_train
   classifier = classifier.fit(counts, targets)
   example_counts = vectorizer.transform(X_test)
   Y_pred = classifier.predict(example_counts)
   print(mdl.__class__.__name__) 
   print(metrics.confusion_matrix(Y_test,Y_pred))
   print(classification_report(Y_test, Y_pred))

AdaBoostClassifier
[[ 234  358]
 [  84 3031]]
              precision    recall  f1-score   support

           0       0.74      0.40      0.51       592
           1       0.89      0.97      0.93      3115

    accuracy                           0.88      3707
   macro avg       0.82      0.68      0.72      3707
weighted avg       0.87      0.88      0.87      3707

DecisionTreeClassifier
[[ 291  301]
 [ 292 2823]]
              precision    recall  f1-score   support

           0       0.50      0.49      0.50       592
           1       0.90      0.91      0.90      3115

    accuracy                           0.84      3707
   macro avg       0.70      0.70      0.70      3707
weighted avg       0.84      0.84      0.84      3707

XGBClassifier
[[ 126  466]
 [  14 3101]]
              precision    recall  f1-score   support

           0       0.90      0.21      0.34       592
           1       0.87      1.00      0.93      3115

    accuracy                           0.87  

**************************Code below are in progress need more time to complete** 

**Word2Vec Model**

In [17]:
import nltk
from gensim.models import word2vec

tokenized_corpus = [nltk.word_tokenize(doc) for doc in list(src['Text'])]

# Set values for various parameters
feature_size = 25    # Word vector dimensionality  
window_context = 20  # Context window size                                                                                    
min_word_count = 1   # Minimum word count                        
sample = 1e-3        # Downsample setting for frequent words
sg = 1               # skip-gram model

w2v_model = word2vec.Word2Vec(tokenized_corpus, size=feature_size, 
                              window=window_context, min_count = min_word_count,
                              sg=sg, sample=sample, iter=10)
w2v_model

In [20]:
w2v_model

In [22]:
w2v_model['great']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.23662362, -0.05379179,  0.01059656, -0.09407976, -0.4382522 ,
       -0.01218937, -0.5350406 ,  0.00562505,  0.19398579,  0.667377  ,
        0.3784096 , -0.1620181 , -0.01792208,  1.1302689 , -0.1320794 ,
        0.14009635,  0.12492268, -0.2001968 ,  0.66609013, -0.38946134,
        0.08393219,  0.21517958, -0.36435458, -0.19027226,  0.4639255 ],
      dtype=float32)

In [23]:
import matplotlib.pyplot as plt

%matplotlib inline

# visualize embeddings
from sklearn.manifold import TSNE

words = w2v_model.wv.index2word
wvs = w2v_model.wv[words]

tsne = TSNE(n_components=2, random_state=42, n_iter=5000, perplexity=5)
np.set_printoptions(suppress=True)
T = tsne.fit_transform(wvs)
labels = words

plt.figure(figsize=(12, 6))
plt.scatter(T[:, 0], T[:, 1], c='orange', edgecolors='r')
for label, x, y in zip(labels, T[:, 0], T[:, 1]):
    plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')

KeyboardInterrupt: ignored

In [30]:
vec_df = pd.DataFrame(wvs, index=words)
vec_df.T

,like,br,good,taste,love,flavor,one,get,make,product,great,use,coffee,try,I,buy,tea,food,would,find,eat,dog,go,really,time,much,well,order,also,amazon,price,give,little,bag,best,think,even,drink,treat,store,...,gallopin,sourish,interresting,tossed,itill,outofstockbr,brownishgreen,bane,gorilla,glucosaminecondroitin,irreplaceable,dishespecially,soupit,findnot,anymorebut,creationnever,stol,weeksvs,analog,117,spansee,mespring,jakemans,passage,highpoints,selecion,dozed,powedered,delghtful,tresures,paula,leisurly,dopio,3499,murphy,softcoated,foilbr,mgsan,cuginos,mealtype
0,-0.253746,-0.023921,-0.125472,-0.217520,-0.236118,-0.411041,-0.206132,-0.149022,-0.294786,-0.229926,-0.236624,-0.454663,0.061904,-0.238438,-0.267096,-0.355451,-0.661807,0.099057,0.001486,-0.150328,-0.033965,-0.004693,-0.142916,-0.159369,-0.356208,-0.192283,-0.248955,-0.396950,-0.192357,-0.186995,-0.168526,-0.059669,-0.253022,-1.034813,-0.232144,-0.057055,-0.145335,-0.061800,-0.038631,-0.480315,...,-0.463642,-0.425778,-0.203089,-0.350726,-0.127325,-0.244004,-0.424705,-0.137257,-0.324762,-0.269358,-0.312484,-0.284200,-0.316388,-0.333320,-0.304466,-0.275182,-0.223868,-0.029690,-0.360891,-0.259224,-0.245214,-0.101824,-0.179955,-0.179042,-0.293455,-0.339531,-0.319140,-0.307639,-0.336789,-0.347460,-0.238953,-0.111318,-0.172039,-0.098327,-0.121612,-0.083967,-0.583626,-0.251431,-0.304721,-0.200947
1,-0.228368,-0.203169,-0.119185,-0.191082,-0.300179,-0.433700,-0.400694,-0.137727,-0.215293,-0.151246,-0.053792,0.156971,-0.869930,-0.223423,-0.141830,-0.239997,-0.590877,-0.060078,-0.363865,0.014244,-0.016019,-0.224318,-0.179005,-0.198238,-0.133393,0.014276,-0.114849,-0.358015,0.049378,-0.240546,-0.178506,-0.352858,-0.169503,-0.302177,-0.338008,-0.311262,-0.001113,-0.248825,-0.387326,-0.001338,...,-0.322267,-0.424265,-0.155515,-0.307474,-0.460885,-0.530015,-0.349928,-0.174522,-0.340120,0.049740,-0.202439,-0.184853,-0.138681,-0.129930,-0.191501,-0.080024,-0.147969,-0.300774,-0.141242,-0.306207,-0.378318,0.052142,-0.024292,-0.045786,-0.483125,-0.507543,-0.167787,-0.172683,-0.262999,-0.287596,-0.295697,-0.294495,-0.202626,-0.063694,-0.091800,-0.081070,-0.162905,-0.031203,-0.212731,-0.324006
2,0.162058,0.201160,0.002860,0.021605,0.329655,0.063133,0.160976,0.281464,0.126699,-0.427509,0.010597,-0.158000,0.273684,-0.122938,-0.002151,0.239586,-0.025266,0.082478,0.105979,0.143810,0.642710,0.349225,0.329557,0.012496,0.152879,0.031263,-0.176816,-0.087702,-0.064475,0.147342,-0.114896,0.076882,0.238222,0.597381,-0.069124,0.178032,0.206949,0.118429,0.592721,0.293886,...,0.260276,0.220655,0.196212,0.318915,0.407209,0.374827,0.026414,0.189704,0.196059,0.224482,0.294870,0.336935,0.276511,0.242635,0.286368,0.215630,0.210215,0.130772,0.171681,0.318446,0.319548,0.103068,0.069152,0.056303,0.118418,0.135722,0.537662,0.585949,0.454784,0.495263,0.303975,0.174479,0.180623,0.264613,0.237248,0.313152,0.400125,0.693048,0.319443,0.244928
3,-0.073390,-0.041289,-0.117190,-0.032473,0.075332,0.001760,0.037176,0.187299,0.037196,0.108361,-0.094080,0.265458,0.789904,-0.110967,-0.069322,0.287884,0.815767,0.402601,0.205381,0.157399,0.019720,0.701259,0.151627,-0.051326,0.125310,0.015034,-0.106804,0.113218,-0.074895,0.329942,0.297086,0.191138,0.074505,0.386261,-0.299055,0.051318,0.201103,0.440059,0.460896,0.524356,...,-0.317844,-0.273581,0.104940,-0.101558,-0.131197,-0.130483,-0.112791,-0.025449,0.070562,0.070522,0.050924,-0.028958,0.077304,0.072570,0.046528,0.029280,0.011200,0.224213,-0.009031,-0.133252,-0.164974,0.119604,0.133350,0.070240,-0.085141,-0.057256,-0.112936,-0.082352,-0.008272,0.017712,0.138504,-0.008421,-0.008372,0.156360,-0.116089,-0.094405,-0.020451,-0.149237,0.062054,0.132828
4,-0.361206,-0.020993,-0.250892,-0.406395,-0.606319,-0.379239,-0.336241,-0.415655,-0.260272,-0.202408,-0.438252,-0.610202,-0.240476,-0.594290,-0.439571,-0.225821,0.144610,0.086808,-0.382644,-0.167879,-0.233687,-0.074581,-0.449351,-0.536740,-0.391242,-0.210363,-0.344268,-0.275424,-0.251443,0.023746,0.202566,-0.620633,-0.331640,0.

In [0]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(vec_df.values)
similarity_df = pd.DataFrame(similarity_matrix, index=words, columns=words)
similarity_df

In [18]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [19]:
!python -m spacy download en_vectors_web_lg

     |████████████████████████████████| 661.8MB 120.0MB/s 
  Created wheel for en-vectors-web-lg: filename=en_vectors_web_lg-2.1.0-cp36-none-any.whl size=663461749 sha256=f9f8522db7fe46fd8eca714939e3d7d5a446e31d467cbf7e282939e8209fc23a
  Stored in directory: /tmp/pip-ephem-wheel-cache-v52hr5jz/wheels/ce/3e/83/59647d0b4584003cce18fb68ecda2866e7c7b2722c3ecaddaf
Successfully built en-vectors-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_vectors_web_lg')


In [26]:
import spacy

nlp = spacy.load('en_vectors_web_lg')
total_vectors = len(nlp.vocab.vectors)

print('Total word vectors:', total_vectors)

OSError: ignored

In [25]:
unique_words = list(set([word for sublist in tokenized_corpus for word in sublist]))

word_glove_vectors = np.array([nlp(word).vector for word in unique_words])
vec_df = pd.DataFrame(word_glove_vectors, index=unique_words)
vec_df

NameError: ignored